**Inference on Blender Bot 2.0**

In [1]:
!pip install torch==1.9.1+cu111 torchvision==0.10.1+cu111 torchaudio===0.9.1 -f https://download.pytorch.org/whl/torch_stable.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |█████████████                   | 834.1 MB 1.3 MB/s eta 0:15:18tcmalloc: large alloc 1147494400 bytes == 0x39d38000 @  0x7f0e72466615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |████████████████▌               | 1055.7 MB 1.3 MB/s eta 0:12:25tcmalloc: large alloc 1434370048 bytes == 0x7e38e000 @  0x7f0e72466615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |█████

In [1]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 8.6 MB/s 
     |████████████████████████████████| 6.6 MB 51.3 MB/s 
     |████████████████████████████████| 596 kB 70.3 MB/s 
     |████████████████████████████████| 86 kB 6.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [8]:
from transformers import BlenderbotTokenizer, BlenderbotForConditionalGeneration

In [9]:
tokenizer = BlenderbotTokenizer.from_pretrained("facebook/blenderbot-400M-distill")

model = BlenderbotForConditionalGeneration.from_pretrained("facebook/blenderbot-400M-distill")

Downloading:   0%|          | 0.00/124k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/61.4k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/16.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.54k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/696M [00:00<?, ?B/s]

In [5]:
sentence = "I want to go the Mars"

In [6]:
inputs = tokenizer(sentence, return_tensors="pt")

In [7]:
results = model.generate(**inputs)

In [8]:
results

tensor([[   1,  281,  422,  913,  287,  425,  287, 7863,  371,  731,   21,  452,
          315,  271, 1205,   20,   83,  305, 2415, 4280,  302,  271, 5680,  305,
         4243, 1228,   21,    2]])

In [9]:
tokenizer.decode(results[0])

'<s> I would love to go to Mars as well. It is the second-largest planet in the Solar System.</s>'

**Fine tuning GPT2 on Custom Dataset**

In [10]:
from transformers import GPT2Tokenizer

In [11]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [12]:
train_path = 'train_dataset.txt'
test_path = 'test_dataset.txt'

In [13]:
from transformers import TextDataset,DataCollatorForLanguageModeling

def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)
     
    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)   
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

train_dataset,test_dataset,data_collator = load_dataset(train_path,test_path,tokenizer)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


In [14]:
from transformers import Trainer, TrainingArguments,AutoModelWithLMHead

model = AutoModelWithLMHead.from_pretrained("gpt2")

training_args = TrainingArguments(
    output_dir="./gpt2-finetuned",
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=8, # batch size for training
    per_device_eval_batch_size=12,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved 
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:925: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


Downloading:   0%|          | 0.00/523M [00:00<?, ?B/s]

In [15]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 25703
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 9639


Step,Training Loss
500,3.658100
1000,2.515400
1500,2.229000
2000,2.083800
2500,1.985800
3000,1.914100
3500,1.826600
4000,1.784200
4500,1.751900
5000,1.727500


Saving model checkpoint to ./gpt2-finetuned/checkpoint-800
Configuration saved in ./gpt2-finetuned/checkpoint-800/config.json
Model weights saved in ./gpt2-finetuned/checkpoint-800/pytorch_model.bin
Saving model checkpoint to ./gpt2-finetuned/checkpoint-1600
Configuration saved in ./gpt2-finetuned/checkpoint-1600/config.json
Model weights saved in ./gpt2-finetuned/checkpoint-1600/pytorch_model.bin
Saving model checkpoint to ./gpt2-finetuned/checkpoint-2400
Configuration saved in ./gpt2-finetuned/checkpoint-2400/config.json
Model weights saved in ./gpt2-finetuned/checkpoint-2400/pytorch_model.bin
Saving model checkpoint to ./gpt2-finetuned/checkpoint-3200
Configuration saved in ./gpt2-finetuned/checkpoint-3200/config.json
Model weights saved in ./gpt2-finetuned/checkpoint-3200/pytorch_model.bin
Saving model checkpoint to ./gpt2-finetuned/checkpoint-4000
Configuration saved in ./gpt2-finetuned/checkpoint-4000/config.json
Model weights saved in ./gpt2-finetuned/checkpoint-4000/pytorch_mod

TrainOutput(global_step=9639, training_loss=1.8952123137576067, metrics={'train_runtime': 3037.1144, 'train_samples_per_second': 25.389, 'train_steps_per_second': 3.174, 'total_flos': 5036991823872000.0, 'train_loss': 1.8952123137576067, 'epoch': 3.0})

In [16]:
trainer.save_model()

Saving model checkpoint to ./gpt2-finetuned
Configuration saved in ./gpt2-finetuned/config.json
Model weights saved in ./gpt2-finetuned/pytorch_model.bin


In [24]:
from transformers import pipeline

generator = pipeline('text-generation', model='./gpt2-finetuned', tokenizer='gpt2')

loading configuration file ./gpt2-finetuned/config.json
Model config GPT2Config {
  "_name_or_path": "./gpt2-finetuned",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "torch_dtype": "float32",
  "transformers_version": "4.19.2",
  "use_cache": true,
  "vocab_size": 50257
}



In [25]:
generator('Zuerst Hähnchen')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Zuerst Hähnchenbrustfilet waschen, trocken schütteln, fein hacken und in einer Pfanne mit Butter anbraten. Fischfilet in Olivenöl anbraten'}]